In [ ]:
!pip install langchain_google_genai langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires

In [2]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader

from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings # r
from langchain.vectorstores import FAISS

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

In [3]:
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from huggingface_hub import login

import os

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

TimeoutException: Requesting secret HUGGINGFACEHUB_API_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
pdf_paths = [f"/content/{filename}" for filename in uploaded.keys()]
print(f"Selected PDFs: {pdf_paths}")

In [ ]:
def get_pdf_text(pdf_docs):
  text = ""
  for pdf in pdf_docs:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      page_text = page.extract_text()
      if page_text:
        text += page_text
  return text

raw_text = get_pdf_text(pdf_paths)
print(f"Length of text: {len(raw_text)}")

In [ ]:
def get_text_chunks(raw_text):
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )

  chunks = text_splitter.split_text(raw_text)
  return chunks

text_chunks = get_text_chunks(raw_text)
print(f"Number of text chunks: {len(text_chunks)}")

In [ ]:
def get_vectorstore(text_chunks):
  embeddings = HuggingFaceEmbeddings(
      model_name="hkunlp/instructor-xl",
      # model_kwargs={"device": "cuda"}
      model_kwargs={"device": "cpu"}
  )

  vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
  return vectorstore

get_vectorstore(text_chunks)


In [ ]:
def get_conversation_chain(vectorstore):
  llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

  return ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=vectorstore.as_retriever(),
      memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
  )

# conversation = get_conversation_chain(get_vectorstore(text_chunks))
chain = get_conversation_chain(get_vectorstore(text_chunks))

In [ ]:
def chat_with_pdf(question):
  response = chain({"question": question})

  for i, msg in enumerate(response["chat_history"]):
    sender = "User" if i % 2 == 0 else "Bot"
    print(f"{sender}: {msg.content}\n")

In [ ]:
chat_with_pdf("Give me a summary of the document.")

In [ ]:
chat_with_pdf("How does diet help in managing diabetes?")

In [ ]:
chat_with_pdf("How does medication help with diabetes?")